<a href="https://colab.research.google.com/github/Shikhasingh12321/DR/blob/main/ViT_and_VGG_final_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
from google.colab import files
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Upload the zip file
uploaded = files.upload(r"C:\Users\Daksh\Desktop\Aptos2019_data.zip")
# Extract the zip file
import io
for file_name, file_obj in uploaded.items():
    with io.BytesIO(file_obj) as zip_buffer:
        with zipfile.ZipFile(zip_buffer) as zip_ref:
          zip_ref.extractall("/content/sample_data")

Saving Aptos2019_data.zip to C:\Users\Daksh\Desktop\Aptos2019_data.zip/Aptos2019_data (1).zip


In [ ]:


# Define the directory path
directory_path = "/content/sample_data"

# Initialize lists to store text data and labels
text_data_list = []
labels_list = []
# Function to get label from file name
def get_label_from_file_name(file_name):
    if file_name.startswith('0_'):
        return '0'
    elif file_name.startswith('1_'):
        return '1'
    elif file_name.startswith('2_'):
        return '2'
    elif file_name.startswith('3_'):
        return '3'
    elif file_name.startswith('4_'):
        return '4'
    else:
        return None
        label = get_label_from_file_name("/content/sample_data/Aptos2019/train.csv")
        labels_list.append(label)   # Replace with actual labels
        print(label)
# Perform feature extraction using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', min_df=5)  # Added stop_words and min_df
text_data_list = [text for text in text_data_list if text.strip()]  # Remove empty documents
if text_data_list:  # Check if list is not empty
    X = vectorizer.fit_transform(text_data_list)
    print("Vocabulary size:", len(vectorizer.vocabulary_))
else:
    print("No text data available.")
import os
import torch
import torchvision
import torchvision.transforms as transforms

class DiabeticRetinopathyDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, label_dir, transform):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_name = self.images[index]
        image_path = os.path.join(self.image_dir, image_name)
        label_path = os.path.join(self.label_dir, image_name + '.csv')
        label = pd.read_csv(label_path).iloc[0, 1]

        image = torchvision.io.read_image(image_path)
        image = self.transform(image)

        return image, label

transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
dataset = DiabeticRetinopathyDataset('/content/sample_data/Aptos2019/train_train', '/content/sample_data/Aptos2019/train.csv', transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import os
import shutil
import torch
import torchvision
import torchvision.transforms as transforms
from transformers import ViTForImageClassification
from torchvision.models import vit_b_16
!pip install torchvision
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.models import vgg19
import torch.optim as optim
model = vit_b_16(pretrained=True)
import timm

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import timm

class LesionDetectionModel(nn.Module):
    def __init__(self):
        super(LesionDetectionModel, self).__init__()
        self.vgg19_features = torchvision.models.vgg19(weights=torchvision.models.VGG19_Weights.DEFAULT).features
        self.vgg19_fc = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 5)
        )
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=5)

    def forward(self, x):
        vgg19_output = self.vgg19_features(x)
        vgg19_output = vgg19_output.view(vgg19_output.size(0), -1)
        vgg19_output = self.vgg19_fc(vgg19_output)

        vit_output = self.vit(x)

        return vgg19_output, vit_output

model = LesionDetectionModel()


# Define the path to the dataset
dataset_root = '/content/sample_data/Aptos2019'

# Define the train and test directories
train_dir = '/content/sample_data/Aptos2019/train_train'
test_dir = '/content/sample_data/Aptos2019/train_test'

# Organize the directory structure
# Create subdirectories for each class within train_dir and test_dir
for dir_path in [train_dir, test_dir]:
    for filename in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, filename)):
            # Extract class label from filename
            if filename.startswith('0_'):
                class_label = 'a'
            elif filename.startswith('1_'):
                class_label = 'b'
            elif filename.startswith('2_'):
                class_label = 'c'
            elif filename.startswith('3_'):
                class_label = 'd'
            elif filename.startswith('4_'):
                class_label = 'e'
            else:
                continue  # Skip files with unrecognized class labels

            # Create class subdirectory if it doesn't exist
            class_dir = os.path.join(dir_path, class_label)
            os.makedirs(class_dir, exist_ok=True)

            # Move the image file into the corresponding class subdirectory
            shutil.move(os.path.join(dir_path, filename), class_dir)

# Define data transforms
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

# Load the dataset
# Use the train_dir directly as it now contains class subdirectories
train_dataset = datasets.ImageFolder(train_dir, transform, is_valid_file=lambda x: x.endswith(('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff')))
test_dataset = datasets.ImageFolder(test_dir, transform, is_valid_file=lambda x: x.endswith(('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff')))


# Define a positive integer value for batch_size
batch_size = 32

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for images, labels in train_loader:
        optimizer.zero_grad()
        vgg19_output, vit_output = model(images)
        loss_vgg19 = criterion(vgg19_output, labels)
        loss_vit = criterion(vit_output, labels)
        loss = loss_vgg19 + loss_vit
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')
    sample_image, _ = next(iter(test_loader))
sample_image = sample_image[0] # Get the first image from the batch
sample_image = sample_image.unsqueeze(0) # Add a batch dimension
# If using GPU, move sample_image to the device
if torch.cuda.is_available():
    sample_image = sample_image.cuda()
    model.cuda()

# Now, pass the image through your LesionDetectionModel
# to get the vgg19_output and vit_output
model = LesionDetectionModel() # Instantiate the LesionDetectionModel
if torch.cuda.is_available():
    model.cuda()
vgg19_output, vit_output = model(sample_image)
vgg19_pred = torch.argmax(vgg19_output)
vit_pred = torch.argmax(vit_output)

print(f"VGG-19 prediction: {vgg19_pred}")
print(f"ViT prediction: {vit_pred}")

No text data available.


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:07<00:00, 72.7MB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create 

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Epoch 1, Loss: 3.3934404850006104
Epoch 2, Loss: 11.195030212402344
Epoch 3, Loss: 5.471508502960205
Epoch 4, Loss: 9.371572494506836
Epoch 5, Loss: 7.838116645812988
Epoch 6, Loss: 4.691323757171631
Epoch 7, Loss: 4.172685623168945
Epoch 8, Loss: 4.329435348510742
Epoch 9, Loss: 3.746964454650879
Epoch 10, Loss: 4.196518898010254
VGG-19 prediction: 0
ViT prediction: 1
